In [13]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [37]:
td=pd.read_excel('flightdata.xlsx')
td.dropna(inplace=True)
airlines=list(td["Airline"].unique())
dep=list(td["Source"].unique())
destination=list(td["Destination"].unique())
td['Journey Day']=pd.to_datetime(td['Date_of_Journey'],format="%d/%m/%Y").dt.day
td['Journey Month']=pd.to_datetime(td['Date_of_Journey'],format="%d/%m/%Y").dt.month
td.drop(["Date_of_Journey"],axis=1,inplace=True)
td["Dep_hour"]=pd.to_datetime(td["Dep_Time"]).dt.hour
td["Dep_min"]=pd.to_datetime(td["Dep_Time"]).dt.minute
td.drop(["Dep_Time"],axis=1,inplace=True)
td["Arrival_hour"]=pd.to_datetime(td["Arrival_Time"]).dt.hour
td["Arrival_min"]=pd.to_datetime(td["Arrival_Time"]).dt.minute
duration=list(td["Duration"])
for i in range(len(duration)):
    if len(duration[i].split()) !=2:
        if "h" in duration[i]:
            duration[i]=duration[i].strip() +" 0m"
        else:
            duration[i]= "0h "+ duration[i]
duration_hours=[]
duration_mins=[]
for i in range(len(duration)):
    duration_hours.append(int(duration[i].split(sep="h")[0]))
    duration_mins.append(int(duration[i].split(sep="m")[0].split()[-1]))
td["Duration_hours"]=duration_hours
td["Duration_mins"]=duration_mins
td.drop(["Duration"],axis=1,inplace=True)
airline=td[["Airline"]]
airline=pd.get_dummies(airline,drop_first=True)
source=td[["Source"]]
source=pd.get_dummies(source,drop_first=True)
des=td[["Destination"]]
des=pd.get_dummies(des,drop_first=True)
td.drop(["Route","Additional_Info"],axis=1,inplace=True)
td.replace({"non-stop":0,"1 stop":1,"2 stops":2,"3 stops":3,"4 stops":4},inplace=True)
stops=list(td['Total_Stops'].unique())
td.drop(["Arrival_Time"],axis=1,inplace=True)
df=pd.concat([td,airline,source,des],axis=1)
df.drop(["Airline","Source","Destination"],axis=1,inplace=True)
columns=df.columns
#=================================================================================================================#

x=df.loc[:,['Total_Stops', 'Journey Day', 'Journey Month', 'Dep_hour',
       'Dep_min', 'Arrival_hour', 'Arrival_min', 'Duration_hours',
       'Duration_mins', 'Airline_Air India', 'Airline_GoAir', 'Airline_IndiGo',
       'Airline_Jet Airways', 'Airline_Jet Airways Business',
       'Airline_Multiple carriers',
       'Airline_Multiple carriers Premium economy', 'Airline_SpiceJet',
       'Airline_Trujet', 'Airline_Vistara', 'Airline_Vistara Premium economy',
       'Source_Chennai', 'Source_Delhi', 'Source_Kolkata', 'Source_Mumbai',
       'Destination_Cochin', 'Destination_Delhi', 'Destination_Hyderabad',
       'Destination_Kolkata', 'Destination_New Delhi']]
y=df["Price"]

#=================================================================================================================#

from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2)

#=================================================================================================================#

from sklearn.ensemble import RandomForestRegressor
reg_rf=RandomForestRegressor(max_depth=14)
print(reg_rf.n_estimators)
reg_rf.fit(x_train,y_train)

#=================================================================================================================#


100


RandomForestRegressor(max_depth=14)

In [38]:
accuracy=reg_rf.score(x_train,y_train)

In [39]:
accuracy

0.9345209616329415

In [40]:
# loading library
import pickle
with open('fpmodel_pkl', 'wb') as files:
    pickle.dump([reg_rf,accuracy,airline,dep,destination,stops,columns], files)